In [1]:
import RnnNetworkModel
import constant as co
import common_resources as cr
torch,device=cr.get_torch_data()
model = RnnNetworkModel.NETWORK(input_dim=co.INPUT_DIM,
            embedding_dim=co.EMBEDDING_DIM,
            hidden_dim=co.HIDDEN_DIM,
            output_dim=co.NUM_CLASSES
)

model = model.to(device=device)
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9) 
clip_value = 1.0
model.parameters

<bound method Module.parameters of NETWORK(
  (embedding): Embedding(1000, 128)
  (rnn): LSTM(128, 256, batch_first=True)
  (rnn1): LSTM(256, 300, batch_first=True)
  (rnn2): LSTM(300, 200, batch_first=True)
  (rnn3): LSTM(200, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=17, bias=True)
  (batch_norm): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)>

In [2]:

data_source=cr.get_data_source()

#get tokenizer object
token=cr.get_tokenizer_object(data_source)

#get encoded input and out put value
input_date,out_put_date=cr.tokenize_input_out_put_field(token=token,data=data_source)

# split into batch
batch_date=cr.batch_spliter(input_data_sources=input_date,out_data_sources=out_put_date)


In [3]:


# from sequence data
import pandas as pd
data_source=cr.get_data_source()

#get tokenizer object
token=cr.get_tokenizer_object(data_source)
seq_data=pd.read_csv('C:\\Toothless\\data\\sequence.csv')
seq_data

input_seq_data,_=cr.tokenize_input_out_put_field(token,seq_data)
def dataConverter(data):
     return list(map(int, data.split("$")))
out_seq_data=[dataConverter(i) for i in seq_data.get("status").values.tolist()]
batch=cr.batch_spliter(input_seq_data,out_seq_data,False)

In [4]:
import random
# batch=batch*100
# random.shuffle(batch)
batch_date=[*batch_date,*batch]

# random.shuffle(batch_date)
# random.shuffle(batch_date)
len(batch_date)


161

In [3]:
def process_hidden_state(data):
        if data is not None:
            return (data[0].to(device),data[1].to(device))

In [5]:
import time
start_time = time.time()
NUM_EPOCHS=3
for epoch in range(NUM_EPOCHS):
    model.train()
    count=0
    hidden_state=None
    for i in batch :
        input,output,sequence=i.get("input"),i.get("output"),i.get("sequence")
        # print(hidden_state)
        if hidden_state is not None:
                hidden_state = [(h.to(device), c.to(device)) for h, c in hidden_state if h is not None and c is not None]
                model.load_hidden_state(hidden_state)
        logits = model(input,sequence)
        # print(output.shape)
        # print(logits.shape)
        loss = criterion(logits, output)       
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        optimizer.step()
        # model.reset_hidden_state()
        # hidden_state=model.save_hidden_state()
        # if i.get("isSeq"):
        #      hidden_state=model.save_hidden_state()
        # else:
        #     model.reset_hidden_state()

        hidden_state=model.save_hidden_state() 
        # model.reset_hidden_state()       
        print (f'Epoch: {epoch+1:03d}/{co.NUM_EPOCHS:03d} | '
                   f'Batch {count:03d}/{len(batch_date):03d} | '
                   f'Loss: {loss:.4f}')
        count+=1
    # model.reset_hidden_state()
    scheduler.step()
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [21]:
batch[127]['output']



tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],
       device='cuda:0')

In [ ]:
batch=cr.batch_spliter(input_seq_data,out_seq_data,True)

In [4]:
#savemodel
torch.save(model.state_dict(), 'model_ini_up_v1_2.pth')

In [8]:
import RnnNetworkModel
import constant as co
import common_resources as cr
torch,device=cr.get_torch_data()
model1 = RnnNetworkModel.NETWORK(input_dim=co.INPUT_DIM,
            embedding_dim=co.EMBEDDING_DIM,
            hidden_dim=co.HIDDEN_DIM,
            output_dim=co.NUM_CLASSES
)

model1 = model1.to(device=device)
optimizer = torch.optim.Adam(model1.parameters(), lr=0.005)
model1.parameters

<bound method Module.parameters of NETWORK(
  (embedding): Embedding(1000, 128)
  (rnn): LSTM(128, 256, batch_first=True)
  (rnn1): LSTM(256, 300, batch_first=True)
  (rnn2): LSTM(300, 200, batch_first=True)
  (rnn3): LSTM(200, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=17, bias=True)
)>

In [ ]:
#load model
model1.load_state_dict(torch.load('C:\\Toothless\\model_ini_up.pth'))

<All keys matched successfully>

In [ ]:
import time
start_time = time.time()
NUM_EPOCHS=50
for epoch in range(NUM_EPOCHS):
    model.train()
    count=0
    hidden_state=None
    for i in batch :
        input,output,sequence=i.get("input"),i.get("output"),i.get("sequence")
        if hidden_state is not None:
            model.load_hidden_state(hidden_state)
        logits = model(input,sequence)
        loss = torch.nn.functional.cross_entropy(logits, output)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        hidden_state=model.save_hidden_state()
        # model.reset_hidden_state()

        
        print (f'Epoch: {epoch+1:03d}/{co.NUM_EPOCHS:03d} | '
                   f'Batch {count:03d}/{len(batch_date):03d} | '
                   f'Loss: {loss:.4f}')
        count+=1
    model.reset_hidden_state()
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')

Epoch: 001/001 | Batch 000/137 | Loss: 264.9913
Epoch: 001/001 | Batch 001/137 | Loss: 180.2625
Epoch: 001/001 | Batch 002/137 | Loss: 113.5064
Epoch: 001/001 | Batch 003/137 | Loss: 70.9202
Epoch: 001/001 | Batch 004/137 | Loss: 68.6320
Epoch: 001/001 | Batch 005/137 | Loss: 56.2609
Epoch: 001/001 | Batch 006/137 | Loss: 49.9164
Epoch: 001/001 | Batch 007/137 | Loss: 47.1169
Epoch: 001/001 | Batch 008/137 | Loss: 39.6987
Epoch: 001/001 | Batch 009/137 | Loss: 42.9246
Epoch: 001/001 | Batch 010/137 | Loss: 39.5247
Epoch: 001/001 | Batch 011/137 | Loss: 39.7180
Epoch: 001/001 | Batch 012/137 | Loss: 36.7565
Epoch: 001/001 | Batch 013/137 | Loss: 39.6622
Epoch: 001/001 | Batch 014/137 | Loss: 38.0344
Epoch: 001/001 | Batch 015/137 | Loss: 35.6606
Epoch: 001/001 | Batch 016/137 | Loss: 39.5027
Epoch: 001/001 | Batch 017/137 | Loss: 37.5552
Epoch: 001/001 | Batch 018/137 | Loss: 38.4259
Epoch: 001/001 | Batch 019/137 | Loss: 38.4520
Epoch: 001/001 | Batch 020/137 | Loss: 35.7009
Epoch: 001